In [1]:
!git clone https://github.com/ai4se-course/ai4se-hse-course-24-25.git

Cloning into 'ai4se-hse-course-24-25'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 3), reused 10 (delta 0), pack-reused 5 (from 1)
Receiving objects: 100% (22/22), 9.60 KiB | 9.60 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
pip install -r /content/ai4se-hse-course-24-25/02-func-name-suggestion/requirements.txt -r /content/ai4se-hse-course-24-25/02-func-name-suggestion/requirements_dev.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.6/542.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1

In [3]:
from datasets import load_dataset

dataset = load_dataset('code_search_net', 'python', split='test', trust_remote_code=True)

dataset = dataset.select(range(1000))

print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 1000
})


In [4]:
from ast import Str
from tree_sitter import Language, Parser
from tree_sitter_python import language

PY_LANGUAGE = Language(language())

# Инициализация парсера
parser = Parser(PY_LANGUAGE)

def extract_function_elements(code: str):
  tree = parser.parse(bytes(code, "utf8"))
  root_node = tree.root_node

  function_name = None
  body_with_comments = []
  body_without_comments = []

  to_delete = []
  for node in root_node.children:
    if node.type == 'function_definition':
        function_name = node.child_by_field_name('name').text.decode('utf8')
        body_node = node.child_by_field_name('body')
        body_with_comments.append(body_node.text.decode('utf8'))
        to_delete += remove_comments_and_strings(body_node)

  # мб костыльно
  code_copy = list(code)
  for start, end in reversed(to_delete):
      code_copy[start:end] = []

  code_copy_str = ''.join(code_copy)
  code_copy_str = '\n'.join([s for s in code_copy_str.splitlines() if not s.isspace()])

  tree = parser.parse(bytes(code_copy_str, "utf8"))
  root_node = tree.root_node
  for node in root_node.children:
    if node.type == 'function_definition':
      body_node = node.child_by_field_name('body')
      body_without_comments.append(body_node.text.decode('utf8'))

  return function_name, ''.join(body_with_comments), ''.join(body_without_comments)

def remove_comments_and_strings(node):
    result = []

    if node.type == 'comment':
      result.append((node.start_byte, node.end_byte))
    elif node.type == 'string':
      if node.parent and node.parent.type == 'expression_statement':
        result.append((node.start_byte, node.end_byte))

    for child in node.children:
      result += remove_comments_and_strings(child)

    return result

In [5]:
# Пример использования
example_code = """
def sina_xml_to_url_list(xml_data):
    \"\"\"str->list
    Convert XML to URL List.
    From Biligrab.
    \"\"\"
    rawurl = []
    # Comment1
    # Comment 2
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]  # Comment 3
        rawurl.append(url.childNodes[0].data)
    return rawurl
"""

function_name, body_with_comments, body_without_comments = extract_function_elements(example_code)
print("Function Name:", function_name)
print("Body with Comments:")
print(body_with_comments)
print("Body without Comments:")
print(body_without_comments)


Function Name: sina_xml_to_url_list
Body with Comments:
"""str->list
    Convert XML to URL List.
    From Biligrab.
    """
    rawurl = []
    # Comment1
    # Comment 2
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]  # Comment 3
        rawurl.append(url.childNodes[0].data)
    return rawurl
Body without Comments:
rawurl = []
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]  
        rawurl.append(url.childNodes[0].data)
    return rawurl


In [6]:
def add_function_elements_to_dataset(dataset):
    dataset = dataset.map(lambda x: {
        'function_name': extract_function_elements(x['whole_func_string'])[0],
        'body_with_comments': extract_function_elements(x['whole_func_string'])[1],
        'body_without_comments': extract_function_elements(x['whole_func_string'])[2]
    })
    return dataset

dataset = add_function_elements_to_dataset(dataset)

Parameter 'function'=<function add_function_elements_to_dataset.<locals>.<lambda> at 0x7e3a13a42dd0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
# Просмотр первых нескольких примеров
for i in range(7):
  print(f"whole_func_string:\n{dataset[i]['whole_func_string']}")
  print(f"function_name:\n{dataset[i]['function_name']}")
  print(f"body_with_comments:\n{dataset[i]['body_with_comments']}")
  print(f"body_without_comments:\n{dataset[i]['body_without_comments']}")
  print('===========================================================')

whole_func_string:
def get_vid_from_url(url):
        """Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')
function_name:
get_vid_from_url
body_with_comments:
"""Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')
body_without_comments:
return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          mat

In [8]:
for example in dataset:
    original_name = example['func_name'].split('.')[-1]
    extracted_name = example['function_name']
    original_doc = example['func_documentation_string'].replace('r"""', '').replace('"""', '').replace("'''", '').strip()
    body_with_comments = example['body_with_comments']
    extracted_doc = body_with_comments.split('"""' if ('"""' in body_with_comments) else "'''")[1].strip()

    assert original_name == extracted_name, f"Mismatch in function name: {original_name} != {extracted_name}"
    assert original_doc == extracted_doc, f"Mismatch in documentation: {original_doc} != {extracted_doc}"

print("All checks passed!")


All checks passed!


In [9]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Salesforce/codet5p-220m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [10]:
torch.cuda.empty_cache()

In [11]:
def predict_function_names(input):
    input_id = tokenizer(input, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    with torch.no_grad():  # Отключение вычисления градиентов
      outputs = model.generate(input_id, max_length=20)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if '(' in prediction:
      prediction = prediction.split('(')[0]
    prediction = prediction.strip()
    if ' ' in prediction:
      splitted = prediction.split(' ')
      prediction = splitted[1] if splitted[0] == 'def' else splitted[0]
    return prediction

In [12]:
import evaluate

exact_match = evaluate.load('exact_match')
rouge = evaluate.load('rouge')

In [13]:
predictions_without_comments = [predict_function_names(f"def <extra_id_0>:\n    {example['body_without_comments']}") for example in dataset]

em_score_wc = exact_match.compute(predictions=predictions_without_comments, references=[example['function_name'] for example in dataset])
rouge_score_wc = rouge.compute(predictions=predictions_without_comments, references=[example['function_name'] for example in dataset])

print("Exact Match Score:", em_score_wc)
print("ROUGE Score:", rouge_score_wc)

Exact Match Score: {'exact_match': 0.153}
ROUGE Score: {'rouge1': 0.3861829365079363, 'rouge2': 0.1928011904761905, 'rougeL': 0.3851968253968251, 'rougeLsum': 0.38379404761904745}


In [14]:
predictions = [predict_function_names(f"  def <extra_id_0>():\n    {example['body_with_comments']}") for example in dataset]

em_score = exact_match.compute(predictions=predictions, references=[example['function_name'] for example in dataset])
rouge_score = rouge.compute(predictions=predictions, references=[example['function_name'] for example in dataset])

print("Exact Match Score:", em_score)
print("ROUGE Score:", rouge_score)

Exact Match Score: {'exact_match': 0.209}
ROUGE Score: {'rouge1': 0.4694119769119771, 'rouge2': 0.26967023809523805, 'rougeL': 0.46717665945165954, 'rougeLsum': 0.46668156565656604}


In [15]:
print("Comparison of Results:")
print("Exact Match Score (Code Only):", em_score_wc)
print("ROUGE Score (Code Only):", rouge_score_wc)
print("Exact Match Score (Code with Docs):", em_score)
print("ROUGE Score (Code with Docs):", rouge_score)

Comparison of Results:
Exact Match Score (Code Only): {'exact_match': 0.153}
ROUGE Score (Code Only): {'rouge1': 0.3861829365079363, 'rouge2': 0.1928011904761905, 'rougeL': 0.3851968253968251, 'rougeLsum': 0.38379404761904745}
Exact Match Score (Code with Docs): {'exact_match': 0.209}
ROUGE Score (Code with Docs): {'rouge1': 0.4694119769119771, 'rouge2': 0.26967023809523805, 'rougeL': 0.46717665945165954, 'rougeLsum': 0.46668156565656604}


In [17]:
# Примеры с наихудшими значениями метрик
worst_examples_em = [(example, prediction) for example, prediction in zip(dataset, predictions) if exact_match.compute(predictions=[prediction], references=[example['function_name']])['exact_match'] == 0]
worst_examples_rouge = [(example, prediction) for example, prediction in zip(dataset, predictions) if rouge.compute(predictions=[prediction], references=[example['function_name']])['rouge1'] == 0]

for example, prediction in worst_examples_em[:5]:
    print("Original Function Name:", example['func_name'])
    print("Predicted Function Name:", prediction)
    print("Function Body:", example['whole_func_string'])
    print("\n")

for example, prediction in worst_examples_rouge[:5]:
    print("Original Function Name:", example['func_name'])
    print("Predicted Function Name:", prediction)
    print("Function Body:", example['whole_func_string'])
    print("\n")


Original Function Name: YouTube.get_vid_from_url
Predicted Function Name: extract_video_id
Function Body: def get_vid_from_url(url):
        """Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')


Original Function Name: sina_xml_to_url_list
Predicted Function Name: parse_xml
Function Body: def sina_xml_to_url_list(xml_data):
    """str->list
    Convert XML to URL List.
    From Biligrab.
    """
    rawurl = []
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]
        rawurl.append(url.childNodes[0].data)
    return rawurl


Original Function Name: makeMimi
Predicted Function Name: packa